# Building a Naïve Bayes Spam Filter

In [13]:
import pandas as pd
import numpy as np
import string
import time

In [14]:
# Load the four dataset files into Python data frames and the two list files into lists
def loadcsv(filepath):
    data_raw = pd.read_csv(filepath, header=0, usecols=[0,1], encoding="latin1")
    data_raw.columns = ['label', 'sms']
    return data_raw

def loadcensored(filepath):
    with open(filepath, 'r') as f:
        return f.read().splitlines()

In [15]:
train = loadcsv('training.csv')
valid = loadcsv('validation.csv')
test1 = loadcsv('test1.csv')
test2 = loadcsv('test2.csv')

cens1 = loadcensored('censored_list_test1.txt')
cens2 = loadcensored('censored_list_test2.txt')

I will start by pre-processing the SMS messages: Remove all punctuation and numbers from the SMS messages, and change all messages to lower case.

In [16]:
def pre_process(df):
    whitelist = set('abcdefghijklmnopqrstuvwxyz ')
    df['sms'] = df.sms.apply(lambda x: ''.join(filter(whitelist.__contains__, x.lower())))
    return df

Before appying the pre-process function, I check the original data first.

In [17]:
# show the first 5 rows of the training data
print(train.head())

  label                                                sms
0   ham  Just sent again. Do you scream and moan in bed...
1   ham            When i have stuff to sell i.ll tell you
2   ham                   Ugh fuck it I'm resubbing to eve
3   ham                      Change windows logoff sound..
4   ham             Ití_ís í«£6 to get in, is that ok?


Now I adopt the pre-process function to the training data.

In [18]:
# pre-process the training data
train = pre_process(train)
# show the first 5 rows of the training data
print(train.head())

  label                                                sms
0   ham  just sent again do you scream and moan in bed ...
1   ham             when i have stuff to sell ill tell you
2   ham                    ugh fuck it im resubbing to eve
3   ham                        change windows logoff sound
4   ham                          its  to get in is that ok


Similarly, I pre-process the other three data files.

In [19]:
valid = pre_process(valid)
test1 = pre_process(test1)
test2 = pre_process(test2)

The functions 'train' and 'train2' calculate and store the prior probabilities and likelihoods. In Naive Bayes this is all the training phase does.

The 'predict' function repeatedly applies Bayes' Theorem to every word in the constructed dictionary, and based on the posterior probability it classifies the message as 'spam' or 'ham'.

The 'score' function calls 'predict' for multiple messages and compares the outcomes with the supplied 'ground truth' labels and thus evaluates the classifier. It also computes and returns a confusion matrix.

The difference between 'train' and 'train2' is that the latter function only takes into account words that are 20 times more likely to appear in a spam message than a ham message. Not all words are equally informative. Using 'train2' will decrease the size of the dictionary significantly, thus making the classifier more efficient. There are multiple other ideas that one can use to construct more informative dictionaries. For example, you can treat words with the same root ('go', 'goes', 'went', 'gone', 'going') as the same word.

---

Now, I will use my training set to train the classifiers 'train' and 'train2'. The training functions require the ham and spam messages to be passed on separately:

In [20]:
hammsgs = train[train['label'] == 'ham']['sms'].tolist()
spammsgs = train[train['label'] == 'spam']['sms'].tolist()
print(len(hammsgs))
print(len(spammsgs))

1826
275


We have 1826 ham messages and 275 spam messages in our training set. Now we can create a classifier and train it:

In [21]:
#define a class 
class NaiveBayesForSpam: 
    #define a method to train the classifier 
    def train(self, hamMessages, spamMessages):
        #create a set of unique words from ham and spam messages
        #make all the words a long string connected with -, split this string into words, and remove duplicates
        self.words = set('-'.join(hamMessages+spamMessages).split())
        #assign [0., 0.] to prior probabilities of ham and spam
        self.priors = np.zeros(2)
        #set the first value as the prior probabilities of ham 
        # P(ham) = num of ham messages / num of all messages
        self.priors[0] = float(len(hamMessages)/(len(spamMessages)+len(hamMessages)))
        #set the second value as the prior probabilities of spam 
        # P(spam) = num of spam messages / num of all messages
        self.priors[1] = 1.0 - self.priors[0]
        #create a list to store likelihoods
        self.likelihoods = []
        
        #iterate the every word in words
        for i, w in enumerate(self.words):
            #likelihood of the word appearing in ham messages = P(word|ham)
            #add 1 for Laplace smoothing to avoid zero probabilities
            prob1 = (1.0 + len([m for m in hamMessages if w in m])) / len(hamMessages)
            #likelihood of the word appearing in spam messages = P(word|spam)
            #add 1 for Laplace smoothing to avoid zero probabilities
            prob2 = (1.0 + len([m for m in spamMessages if w in m])) / len(spamMessages)
            #limit the max likelihood to be 0.95
            #each element in likelihoods is [P(word|ham), P(word|spam)]
            self.likelihoods.append([min(prob1, 0.95), min(prob2, 0.95)])
            #convert list likelihoods to array, transpose so that each row corresponds to a class
        self.likelihoods = np.array(self.likelihoods).T
    
    #alternative training method                 
    def train2(self, hamMessages, spamMessages):
        #same as train
        self.words = set('-'.join(hamMessages+spamMessages).split())
        self.priors = np.zeros(2)
        self.priors[0] = float(len(hamMessages)/(len(spamMessages)+len(hamMessages)))
        self.priors[1] = 1.0 - self.priors[0]
        self.likelihoods = []
        #create a list to store spam words
        spamkeywords = []
        
        #interate every word in words to calculate the likelihoods of the word in ham and spam                       
        for i, w in enumerate(self.words):
            prob1 = (1.0 + len([m for m in hamMessages if w in m])) / len(hamMessages) #P(word|ham)
            prob2 = (1.0 + len([m for m in spamMessages if w in m])) / len(spamMessages) #P(word|spam)
            #add a condition
            #if the word is much more likely to appear in spam
            if prob1*20 <prob2:
                self.likelihoods.append([min(prob1, 0.95), min(prob2, 0.95)])
                spamkeywords.append(w) #append that word to spamkeywords
        #update words to include only spamkeywords
        self.words = spamkeywords
        #likellihoods now include only filtered words (more likely in spam)
        self.likelihoods = np.array(self.likelihoods).T
    
    #define a method to make predictions
    def predict(self, message):
        #set the initial posterior probailities  as prior probabilities 
        posteriors = np.copy(self.priors)
        #iterate every word in words
        for i, w in enumerate(self.words):
            #if w is found in the message(case-insensitive)
            if w in message.lower(): 
                #updated posterior = posterior*likelihood
                posteriors*= self.likelihoods[:, i]
            else:
                #if w is not found in message, updated posterior = posterior*(1-likelihood)
                posteriors*= np.ones(2) - self.likelihoods[:, i]
            posteriors = posteriors/np.linalg.norm(posteriors, ord = 1) #normalise 
        if posteriors[0] > 0.5:
            return ['ham', posteriors[0]] #likely to be ham
        return ['spam', posteriors[1]] #likely to be spam
    
    #define a method to measure the accuracies of the classifier 
    def score(self, messages, labels):
        #initialize a confusion matrix
        confusion = np.zeros(4).reshape(2,2)
        #iterate every word in messages and give scores to confusion matrix
        for m, l in zip(messages, labels):
            #prediction is ham, actual label is ham
            if self.predict(m)[0] == 'ham' and l == 'ham':
                confusion[0,0]+=1
            #prediction is ham, actual label is spam
            elif self.predict(m)[0] == 'ham' and l == 'spam':
                confusion[0,1]+=1
            #prediction is spam, actual label is ham
            elif self.predict(m)[0] == 'spam' and l == 'ham':
                confusion[1,0]+=1
            #prediction is spam, actual label is spam
            elif self.predict(m)[0] == 'spam' and l == 'spam':
                confusion[1,1]+=1
        #return accuracy score = num of words correctly classified / num of all words , and the confusion matrix
        return (confusion[0,0] + confusion[1,1]) / float(confusion.sum()), confusion

In [22]:
clf = NaiveBayesForSpam()

In [23]:
clf.train(hammsgs, spammsgs)
clf.train2(hammsgs, spammsgs)

We employ both training methods to get two separate classifiers.

In [24]:
clf = NaiveBayesForSpam()
time1 = time.time()
clf.train(hammsgs, spammsgs)
accuracy_train, confusion_matrix_train = clf.score(train['sms'], train['label'])
time2 = time.time()
print('For train function:')
print('In-sample accuracy is {}'.format(accuracy_train))
print('In-sample confusion matrix is \n{}'.format(confusion_matrix_train))
print("Run time: {}".format(round(time2 - time1, 2)))

For train function:
In-sample accuracy is 0.96573060447406
In-sample confusion matrix is 
[[1795.   41.]
 [  31.  234.]]
Run time: 98.57


In [25]:
clf2 = NaiveBayesForSpam()
time1 = time.time()
clf2.train2(hammsgs, spammsgs)
accuracy2_train, confusion_matrix2_train = clf2.score(train['sms'], train['label'])
time2 = time.time()
print('For train2 function:')
print('In-sample accuracy is {}'.format(accuracy2_train))
print('In-sample confusion matrix is \n{}'.format(confusion_matrix2_train))
print("Run time: {}".format(round(time2 - time1, 2)))

For train2 function:
In-sample accuracy is 0.9690623512613041
In-sample confusion matrix is 
[[1819.   58.]
 [   7.  217.]]
Run time: 3.08


Using the validation set, I will explore how each of the two classifiers performs out of sample:

In [26]:
time1 = time.time()
accuracy_valid, confusion_matrix_valid = clf.score(valid['sms'], valid['label'])
time2 = time.time()
print('For train function:')
print('Out-of-sample accuracy is {}'.format(accuracy_valid))
print('Out-of-sample confusion matrix is \n{}'.format(confusion_matrix_valid))
print("Run time: {}".format(round(time2 - time1, 2)))

For train function:
Out-of-sample accuracy is 0.9655172413793104
Out-of-sample confusion matrix is 
[[778.  21.]
 [ 10.  90.]]
Run time: 38.08


In [27]:
time1 = time.time()
accuracy2_valid, confusion_matrix2_valid = clf2.score(valid['sms'], valid['label'])
time2 = time.time()
print('For train2 function:')
print('Out-of-sample accuracy is {}'.format(accuracy2_valid))
print('Out-of-sample confusion matrix is \n{}'.format(confusion_matrix2_valid))
print("Run time: {}".format(round(time2 - time1, 2)))

For train2 function:
Out-of-sample accuracy is 0.9632925472747497
Out-of-sample confusion matrix is 
[[784.  29.]
 [  4.  82.]]
Run time: 1.02


The difference between 'train' and 'train2' is that the latter function only takes into account words that are 20 times more likely to appear in a spam message than a ham message. This makes the dictionary much smaller and hence speeds up the algorithm -> 'train2' classifier is faster

The reason why a simpler model (less words) give a better performance (not just out-of-sample (in the validation set), where overfitting may deteriorate the quality of more sophisticated methods, but also in-sample (in the training set)) is the conditional independence assumption of Naive Bayes. With more words in our dictionary, the number of correlated words is larger and therefore the violation of our theoretical assumption stronger. It appears that 'train2' mediates this effect -> 'train2' classifier has a better accuracy noth on the training and validation set

We can see from the confusion matrices that the number of FPs (ham messages classified as spam messages) for the 'train' function is 21. As for 'train2' function, the number of FPs is 29.

Reduce false positives at the expense of possibly having more false negatives (spam messages classified as ham messages) -> We can achieve the trade-off between FPs and FNs by changing the following line in the 'predict' function: 

In [ ]:
# if posteriors[0] > 0.5:
#     return ['ham', posteriors[0]]

By modifying the threshold from 0.5 to other values. To decrease the FP rate, we could decrease the threshold.

---

Now, I am going to use the test sets and lists of censored words. 10% and 30% of the keywords obtained from the 'train' function have been removed ("censored") from the messages in *test1* and *test2* respectively, therefore, I have no knowledge whether the test sets contain these words or not. The censored words, listed in the *.txt* files, must then be treated as missing values.

I will modify the 'predict' function in the code to implement the change and use it to report accuracies on *test1*, with both 'train' and 'train2'.

In [28]:
class NaiveBayesForSpamCensored(NaiveBayesForSpam):
    def predictcensored(self, message, censored_words):
        posterriors = np.copy(self.priors)
        for i, w in enumerate(self.words):
            if w in censored_words: # do nothing
                continue
            elif w in message.lower():
                posterriors *= self.likelihoods[:, i]
            else:
                posterriors *= np.ones(2) - self.likelihoods[:, i]
            posterriors = posterriors / np.linalg.norm(posterriors) # normalise
        if posterriors[0] > 0.5:
            return ['ham', posterriors[0]]
        return ['spam', posterriors[1]]
    
    def scorecensored(self, messages, labels, censored_words):
        confusion = np.zeros(4).reshape(2, 2)
        for m, l in zip(messages, labels):
            if self.predictcensored(m, censored_words)[0] == 'ham' and l == 'ham':
                confusion[0, 0] += 1
            elif self.predictcensored(m, censored_words)[0] == 'ham' and l == 'spam':
                confusion[0, 1] += 1
            elif self.predictcensored(m, censored_words)[0] == 'spam' and l == 'ham':
                confusion[1, 0] += 1
            elif self.predictcensored(m, censored_words)[0] == 'spam' and l == 'spam':
                confusion[1, 1] += 1
        return (confusion[0, 0] + confusion[1, 1]) / float(confusion.sum()), confusion

In [29]:
clf_censored = NaiveBayesForSpamCensored()
time1 = time.time()
clf_censored.train(hammsgs, spammsgs)
accuracy_censored_test1, confusion_matrix_censored_test1 = clf_censored.scorecensored(test1['sms'], test1['label'], cens1)
time2 = time.time()
print('For train function with censored words:')
print('Test1 accuracy is {}'.format(accuracy_censored_test1))
print('Test1 confusion matrix is \n{}'.format(confusion_matrix_censored_test1))
print("Run time: {}".format(round(time2 - time1, 2)))

For train function with censored words:
Test1 accuracy is 0.9672897196261683
Test1 confusion matrix is 
[[1101.   28.]
 [  14.  141.]]
Run time: 109.8


In [30]:
clf_censored2 = NaiveBayesForSpamCensored()
time1 = time.time()
clf_censored2.train2(hammsgs, spammsgs)
accuracy_censored2_test1, confusion_matrix_censored2_test1 = clf_censored2.scorecensored(test1['sms'], test1['label'], cens1)
time2 = time.time()
print('For train2 function with censored words:')
print('Test1 accuracy is {}'.format(accuracy_censored2_test1))
print('Test1 confusion matrix is \n{}'.format(confusion_matrix_censored2_test1))
print("Run time: {}".format(round(time2 - time1, 2)))

For train2 function with censored words:
Test1 accuracy is 0.9688473520249221
Test1 confusion matrix is 
[[1113.   38.]
 [   2.  131.]]
Run time: 3.63


I repeat the process using test2.

In [31]:
clf_censored = NaiveBayesForSpamCensored()
time1 = time.time()
clf_censored.train(hammsgs, spammsgs)
accuracy_censored_test2, confusion_matrix_censored_test2 = clf_censored.scorecensored(test2['sms'], test2['label'], cens2)
time2 = time.time()
print('For train function with censored words:')
print('Test2 accuracy is {}'.format(accuracy_censored_test2))
print('Test2 confusion matrix is \n{}'.format(confusion_matrix_censored_test2))
print("Run time: {}".format(round(time2 - time1, 2)))

For train function with censored words:
Test2 accuracy is 0.9611197511664075
Test2 confusion matrix is 
[[1090.   46.]
 [   4.  146.]]
Run time: 384.05


In [32]:
clf_censored2 = NaiveBayesForSpamCensored()
time1 = time.time()
clf_censored2.train2(hammsgs, spammsgs)
accuracy_censored2_test2, confusion_matrix_censored2_test2 = clf_censored2.scorecensored(test2['sms'], test2['label'], cens2)
time2 = time.time()
print('For train2 function with censored words:')
print('Test2 accuracy is {}'.format(accuracy_censored2_test2))
print('Test2 confusion matrix is \n{}'.format(confusion_matrix_censored2_test2))
print("Run time: {}".format(round(time2 - time1, 2)))

For train2 function with censored words:
Test2 accuracy is 0.9618973561430794
Test2 confusion matrix is 
[[1092.   47.]
 [   2.  145.]]
Run time: 15.15


Performance gap between classifier trained using 'train2' and that using 'train' decreases for both *test1* and *test2* with respect to both accuracy and run time (from 20x to 17x).
The result implies that randomly dropping some of the features may mitigate the violation of the conditional independence assumption and simplify the computation.